In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
pd.set_option('display.max_columns', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/microsoft-adventure-works-cycles-customer-data/AWCustomers.csv
/kaggle/input/microsoft-adventure-works-cycles-customer-data/AWTest-Classification.csv
/kaggle/input/microsoft-adventure-works-cycles-customer-data/AWSales.csv
/kaggle/input/microsoft-adventure-works-cycles-customer-data/AWTest-Regression.csv


In [2]:
df_customer=pd.read_csv("../input/microsoft-adventure-works-cycles-customer-data/AWCustomers.csv")
df_sales=pd.read_csv("../input/microsoft-adventure-works-cycles-customer-data/AWSales.csv")

In [3]:
df_customer.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'LastUpdated'],
      dtype='object')

In [4]:
df_sales.columns

Index(['CustomerID', 'BikeBuyer', 'AvgMonthSpend'], dtype='object')

In [5]:
df=pd.concat([df_customer,df_sales],axis=1)

In [6]:
df.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'LastUpdated', 'CustomerID', 'BikeBuyer',
       'AvgMonthSpend'],
      dtype='object')

Number of columns in the dataset is:27

In [7]:
df.shape[1]


27

In [8]:
df.drop(['CustomerID'],axis=1,inplace=True)

In [9]:
df.head()

,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend
0,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,Australia,2500,1 (11) 500 555-0115,1987-11-13,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06,1.0,50.97
1,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,Canada,V9B 2C3,787-555-0136,1972-07-21,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06,1.0,53.11
2,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,United States,91791,120-555-0111,1985-11-09,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06,1.0,54.08
3,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,United Kingdom,L4 4HB,1 (11) 500 555-0167,1977-10-18,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06,1.0,56.93
4,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,Germany,59368,1 (11) 500 555-0113,1975-02-05,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06,1.0,55.41


In [10]:
df.tail()

,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend
18356,NaN,Grace,C,Bailey,NaN,6627 Camelback Ct.,NaN,Coronado,California,United States,92118,292-555-0130,1990-11-11,Graduate Degree,Skilled Manual,F,M,0,1,0,2,52953,2017-03-06,NaN,NaN
18357,NaN,Tasha,NaN,Deng,NaN,9627 Kendall Rd,NaN,Port Macquarie,New South Wales,Australia,2444,1 (11) 500 555-0171,1992-10-13,Bachelors,Skilled Manual,F,S,0,2,0,0,60992,2017-03-06,NaN,NaN
18358,NaN,Jaclyn,NaN,Zhang,NaN,4364 Viera Avenue,NaN,Beaverton,Oregon,United States,97005,819-555-0110,1983-11-24,Partial College,Skilled Manual,F,S,0,2,0,0,51859,2017-03-06,NaN,NaN
18359,NaN,Erin,I,Reed,NaN,3866 Mt. Everest Court,NaN,Vancouver,British Columbia,Canada,V7L 4J4,791-555-0118,1995-06-15,High School,Clerical,F,S,0,0,0,0,87177,2017-03-06,NaN,NaN
18360,NaN,Amanda,NaN,Perry,NaN,7870 Orangewood Dr.,NaN,Grossmont,California,United States,91941,553-555-0122,1986-03-26,Graduate Degree,Professional,F,M,1,2,0,0,136856,2017-03-06,NaN,NaN


In [11]:
df.drop(['Title','FirstName','MiddleName','LastName','Suffix','AddressLine2'],axis=1,inplace=True)

In [12]:
df.drop(['PhoneNumber'],axis=1,inplace=True)

In [13]:
df.drop(['PostalCode'],axis=1,inplace=True)

In [14]:
df.columns.shape

(17,)

In [15]:
df.describe()

,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer,AvgMonthSpend
count,18361.000000,18361.000000,18361.000000,18361.000000,18361.000000,18355.00000,18355.000000
mean,0.610588,1.270301,0.338162,0.850389,72754.779642,0.55173,51.767207
std,0.487630,0.913989,0.568957,0.927315,30686.014313,0.49733,3.438024
min,0.000000,0.000000,0.000000,0.000000,25435.000000,0.00000,44.100000
25%,0.000000,1.000000,0.000000,0.000000,53312.000000,0.00000,49.410000
50%,1.000000,1.000000,0.000000,0.000000,61851.000000,1.00000,51.420000
75%,1.000000,2.000000,1.000000,2.000000,87410.000000,1.00000,53.600000
max,1.000000,5.000000,3.000000,3.000000,139115.000000,1.00000,65.290000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AddressLine1          18361 non-null  object 
 1   City                  18361 non-null  object 
 2   StateProvinceName     18361 non-null  object 
 3   CountryRegionName     18361 non-null  object 
 4   BirthDate             18361 non-null  object 
 5   Education             18361 non-null  object 
 6   Occupation            18361 non-null  object 
 7   Gender                18361 non-null  object 
 8   MaritalStatus         18361 non-null  object 
 9   HomeOwnerFlag         18361 non-null  int64  
 10  NumberCarsOwned       18361 non-null  int64  
 11  NumberChildrenAtHome  18361 non-null  int64  
 12  TotalChildren         18361 non-null  int64  
 13  YearlyIncome          18361 non-null  int64  
 14  LastUpdated           18361 non-null  object 
 15  BikeBuyer          

In [17]:
df.BirthDate=pd.to_datetime(df.BirthDate)


In [18]:
df.BirthDate

0       1987-11-13
1       1972-07-21
2       1985-11-09
3       1977-10-18
4       1975-02-05
           ...    
18356   1990-11-11
18357   1992-10-13
18358   1983-11-24
18359   1995-06-15
18360   1986-03-26
Name: BirthDate, Length: 18361, dtype: datetime64[ns]

In [19]:
df.LastUpdated=pd.to_datetime(df.LastUpdated)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   AddressLine1          18361 non-null  object        
 1   City                  18361 non-null  object        
 2   StateProvinceName     18361 non-null  object        
 3   CountryRegionName     18361 non-null  object        
 4   BirthDate             18361 non-null  datetime64[ns]
 5   Education             18361 non-null  object        
 6   Occupation            18361 non-null  object        
 7   Gender                18361 non-null  object        
 8   MaritalStatus         18361 non-null  object        
 9   HomeOwnerFlag         18361 non-null  int64         
 10  NumberCarsOwned       18361 non-null  int64         
 11  NumberChildrenAtHome  18361 non-null  int64         
 12  TotalChildren         18361 non-null  int64         
 13  YearlyIncome    

In [21]:
var_cat=[]
var_num=[]
for c in df.columns:
    if df[c].dtype=='object':
        var_cat.append(c)
    if (df[c].dtype=='int64'or df[c].dtype=='float64'):
         var_num.append(c)
            
        
 
    

In [22]:
print("Categorical Variables: ",var_cat)


Categorical Variables:  ['AddressLine1', 'City', 'StateProvinceName', 'CountryRegionName', 'Education', 'Occupation', 'Gender', 'MaritalStatus']


In [23]:
cat_ordinal=[]
cat_nominal=[]
for c in var_cat:
    if (df[c].nunique()) >5:
        cat_nominal.append(c)
    else:
        cat_ordinal.append(c)


In [24]:
cat_nominal


['AddressLine1', 'City', 'StateProvinceName', 'CountryRegionName']

In [25]:
cat_ordinal

['Education', 'Occupation', 'Gender', 'MaritalStatus']

Education is truly ordinal, Occupation should be in nominal and Gender,MaritalStatus can be OHE

In [26]:
num_discrete=[]
num_continuous=[]
for c in var_num:
    if (df[c].nunique()) >10:
        num_continuous.append(c)
    else:
        num_discrete.append(c)

In [27]:
num_discrete

['HomeOwnerFlag',
 'NumberCarsOwned',
 'NumberChildrenAtHome',
 'TotalChildren',
 'BikeBuyer']

In [28]:
num_continuous


['YearlyIncome', 'AvgMonthSpend']

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   AddressLine1          18361 non-null  object        
 1   City                  18361 non-null  object        
 2   StateProvinceName     18361 non-null  object        
 3   CountryRegionName     18361 non-null  object        
 4   BirthDate             18361 non-null  datetime64[ns]
 5   Education             18361 non-null  object        
 6   Occupation            18361 non-null  object        
 7   Gender                18361 non-null  object        
 8   MaritalStatus         18361 non-null  object        
 9   HomeOwnerFlag         18361 non-null  int64         
 10  NumberCarsOwned       18361 non-null  int64         
 11  NumberChildrenAtHome  18361 non-null  int64         
 12  TotalChildren         18361 non-null  int64         
 13  YearlyIncome    

BirthDate and Last Updated are datetime features

In [30]:
df.shape

(18361, 17)

In [31]:
df.isnull().sum()

AddressLine1            0
City                    0
StateProvinceName       0
CountryRegionName       0
BirthDate               0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
LastUpdated             0
BikeBuyer               6
AvgMonthSpend           6
dtype: int64

In [32]:
df[df['BikeBuyer'].isnull()]

,AddressLine1,City,StateProvinceName,CountryRegionName,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend
18355,8527 Rock Oak Road,Melton,Victoria,Australia,1967-08-20,High School,Manual,M,M,1,1,1,2,30442,2017-03-06,NaN,NaN
18356,6627 Camelback Ct.,Coronado,California,United States,1990-11-11,Graduate Degree,Skilled Manual,F,M,0,1,0,2,52953,2017-03-06,NaN,NaN
18357,9627 Kendall Rd,Port Macquarie,New South Wales,Australia,1992-10-13,Bachelors,Skilled Manual,F,S,0,2,0,0,60992,2017-03-06,NaN,NaN
18358,4364 Viera Avenue,Beaverton,Oregon,United States,1983-11-24,Partial College,Skilled Manual,F,S,0,2,0,0,51859,2017-03-06,NaN,NaN
18359,3866 Mt. Everest Court,Vancouver,British Columbia,Canada,1995-06-15,High School,Clerical,F,S,0,0,0,0,87177,2017-03-06,NaN,NaN
18360,7870 Orangewood Dr.,Grossmont,California,United States,1986-03-26,Graduate Degree,Professional,F,M,1,2,0,0,136856,2017-03-06,NaN,NaN


In [33]:
(df.isnull().sum()/df.shape[0])*100

AddressLine1            0.000000
City                    0.000000
StateProvinceName       0.000000
CountryRegionName       0.000000
BirthDate               0.000000
Education               0.000000
Occupation              0.000000
Gender                  0.000000
MaritalStatus           0.000000
HomeOwnerFlag           0.000000
NumberCarsOwned         0.000000
NumberChildrenAtHome    0.000000
TotalChildren           0.000000
YearlyIncome            0.000000
LastUpdated             0.000000
BikeBuyer               0.032678
AvgMonthSpend           0.032678
dtype: float64

As the null values are less than 20%, we can safely remove them.


In [34]:
df.drop(index=df[df['BikeBuyer'].isnull()].index,axis=0,inplace=True)

In [35]:
df.isnull().sum()

AddressLine1            0
City                    0
StateProvinceName       0
CountryRegionName       0
BirthDate               0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
LastUpdated             0
BikeBuyer               0
AvgMonthSpend           0
dtype: int64

In [36]:
num_continuous

['YearlyIncome', 'AvgMonthSpend']

In [37]:
# Normalizing by min max scaler
scaler=MinMaxScaler()
df[['YearlyIncome_normalized','AvgMonthSpend_normalized']]=scaler.fit_transform(df[num_continuous])

In [38]:
# Normalizing by Z score
stdscaler=StandardScaler()

df[['YearlyIncome_std','AvgMonthSpend_std']]=stdscaler.fit_transform(df[num_continuous])

In [39]:
df['Male']=pd.get_dummies(df['Gender'],drop_first=True)

In [40]:
df['Single']=pd.get_dummies(df['MaritalStatus'],drop_first=True)

In [41]:
df.drop(['Gender','MaritalStatus'],axis=1,inplace=True)

In [42]:
df

,AddressLine1,City,StateProvinceName,CountryRegionName,BirthDate,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend,YearlyIncome_normalized,AvgMonthSpend_normalized,YearlyIncome_std,AvgMonthSpend_std,Male,Single
0,7090 C. Mount Hood,Wollongong,New South Wales,Australia,1987-11-13,Bachelors,Clerical,1,3,0,1,81916,2017-03-06,1.0,50.97,0.496842,0.324210,0.298539,-0.231886,1,0
1,3651 Willow Lake Rd,Shawnee,British Columbia,Canada,1972-07-21,Partial College,Clerical,1,2,1,2,81076,2017-03-06,1.0,53.11,0.489453,0.425201,0.271163,0.390582,1,0
2,1774 Tice Valley Blvd.,West Covina,California,United States,1985-11-09,Bachelors,Clerical,0,3,0,0,86387,2017-03-06,1.0,54.08,0.536172,0.470977,0.444250,0.672728,0,1
3,2103 Baldwin Dr,Liverpool,England,United Kingdom,1977-10-18,Partial College,Skilled Manual,1,2,1,2,61481,2017-03-06,1.0,56.93,0.317083,0.605474,-0.367445,1.501715,1,0
4,Am Gallberg 234,Werne,Nordrhein-Westfalen,Germany,1975-02-05,Partial College,Skilled Manual,1,1,0,0,51804,2017-03-06,1.0,55.41,0.231958,0.533742,-0.682822,1.059589,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18350,3469 Relief Valley Ct.,Royal Oak,British Columbia,Canada,1960-09-06,Bachelors,Clerical,1,1,0,2,87320,2017-03-06,0.0,48.14,0.544379,0.190656,0.474657,-1.055055,0,0
18351,6126 North Sixth Street,Redmond,Washington,United States,1986-08-15,Graduate Degree,Clerical,0,1,0,0,84374,2017-03-06,1.0,50.23,0.518464,0.289287,0.378646,-0.447131,0,1
18352,7182 Olive Hill,Cliffside,British Columbia,Canada,1986-11-14,Graduate Degree,Skilled Manual,0,0,0,0,50961,2017-03-06,1.0,49.56,0.224543,0.257669,-0.710296,-0.642016,0,1
18353,"487, rue Montcalm",Les Ulis,Essonne,France,1983-02-05,Partial High School,Manual,0,1,0,0,33678,2017-03-06,0.0,51.42,0.072511,0.345446,-1.273555,-0.100993,0,1


In [43]:
len(df['YearlyIncome'].value_counts().index)

15351

#### From the above cell it is quite clear that if we plot the data as it is the plot will be unreadable,the technique of binning will be highly helpful for us.

#### pd.cut(), will actually help us to divide the this huge series into several bins.

#### Features such as YearlyIncome and AvgMonthSpend are perfect for binning


In [44]:
YearlyIncome_min=df['YearlyIncome'].min()
YearlyIncome_max=df['YearlyIncome'].max()
YearlyIncome_max-YearlyIncome_min


113680

In [45]:
bin_YearlyIncome=np.arange(YearlyIncome_min,YearlyIncome_max,step=20000)
# arrange has stepsize whereas linspace has the length of the number of divisions
len(bin_YearlyIncome)
bin_YearlyIncome
YearlyIncome_label=['below low','low','Medium','above medium','High']

In [46]:
df['YearlyBinned']=pd.cut(df['YearlyIncome'],bins=bin_YearlyIncome,labels=YearlyIncome_label)

In [47]:
df

,AddressLine1,City,StateProvinceName,CountryRegionName,BirthDate,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend,YearlyIncome_normalized,AvgMonthSpend_normalized,YearlyIncome_std,AvgMonthSpend_std,Male,Single,YearlyBinned
0,7090 C. Mount Hood,Wollongong,New South Wales,Australia,1987-11-13,Bachelors,Clerical,1,3,0,1,81916,2017-03-06,1.0,50.97,0.496842,0.324210,0.298539,-0.231886,1,0,Medium
1,3651 Willow Lake Rd,Shawnee,British Columbia,Canada,1972-07-21,Partial College,Clerical,1,2,1,2,81076,2017-03-06,1.0,53.11,0.489453,0.425201,0.271163,0.390582,1,0,Medium
2,1774 Tice Valley Blvd.,West Covina,California,United States,1985-11-09,Bachelors,Clerical,0,3,0,0,86387,2017-03-06,1.0,54.08,0.536172,0.470977,0.444250,0.672728,0,1,above medium
3,2103 Baldwin Dr,Liverpool,England,United Kingdom,1977-10-18,Partial College,Skilled Manual,1,2,1,2,61481,2017-03-06,1.0,56.93,0.317083,0.605474,-0.367445,1.501715,1,0,low
4,Am Gallberg 234,Werne,Nordrhein-Westfalen,Germany,1975-02-05,Partial College,Skilled Manual,1,1,0,0,51804,2017-03-06,1.0,55.41,0.231958,0.533742,-0.682822,1.059589,1,1,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18350,3469 Relief Valley Ct.,Royal Oak,British Columbia,Canada,1960-09-06,Bachelors,Clerical,1,1,0,2,87320,2017-03-06,0.0,48.14,0.544379,0.190656,0.474657,-1.055055,0,0,above medium
18351,6126 North Sixth Street,Redmond,Washington,United States,1986-08-15,Graduate Degree,Clerical,0,1,0,0,84374,2017-03-06,1.0,50.23,0.518464,0.289287,0.378646,-0.447131,0,1,Medium
18352,7182 Olive Hill,Cliffside,British Columbia,Canada,1986-11-14,Graduate Degree,Skilled Manual,0,0,0,0,50961,2017-03-06,1.0,49.56,0.224543,0.257669,-0.710296,-0.642016,0,1,low
18353,"487, rue Montcalm",Les Ulis,Essonne,France,1983-02-05,Partial High School,Manual,0,1,0,0,33678,2017-03-06,0.0,51.42,0.072511,0.345446,-1.273555,-0.100993,0,1,below low


In [48]:
AvgMonthSpend_min=df['AvgMonthSpend'].min()
AvgMonthSpend_max=df['AvgMonthSpend'].max()
AvgMonthSpend_max-AvgMonthSpend_min

21.190000000000005

In [49]:
bin_AvgMonthSpend=np.arange(AvgMonthSpend_min,AvgMonthSpend_max,step=5)
# arrange has stepsize whereas linspace has the length of the number of divisions
print(bin_AvgMonthSpend)
print('length of AvgMonthSpend:',len(bin_AvgMonthSpend))

AvgMonthSpend_label=['Low','Medium','Above Medium','High']

[44.1 49.1 54.1 59.1 64.1]
length of AvgMonthSpend: 5


In [50]:
# pd.cut()
df['AvgMonthSpendBinned']=pd.cut(df['AvgMonthSpend'],bins=bin_AvgMonthSpend,labels=AvgMonthSpend_label)

In [51]:
df

,AddressLine1,City,StateProvinceName,CountryRegionName,BirthDate,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend,YearlyIncome_normalized,AvgMonthSpend_normalized,YearlyIncome_std,AvgMonthSpend_std,Male,Single,YearlyBinned,AvgMonthSpendBinned
0,7090 C. Mount Hood,Wollongong,New South Wales,Australia,1987-11-13,Bachelors,Clerical,1,3,0,1,81916,2017-03-06,1.0,50.97,0.496842,0.324210,0.298539,-0.231886,1,0,Medium,Medium
1,3651 Willow Lake Rd,Shawnee,British Columbia,Canada,1972-07-21,Partial College,Clerical,1,2,1,2,81076,2017-03-06,1.0,53.11,0.489453,0.425201,0.271163,0.390582,1,0,Medium,Medium
2,1774 Tice Valley Blvd.,West Covina,California,United States,1985-11-09,Bachelors,Clerical,0,3,0,0,86387,2017-03-06,1.0,54.08,0.536172,0.470977,0.444250,0.672728,0,1,above medium,Medium
3,2103 Baldwin Dr,Liverpool,England,United Kingdom,1977-10-18,Partial College,Skilled Manual,1,2,1,2,61481,2017-03-06,1.0,56.93,0.317083,0.605474,-0.367445,1.501715,1,0,low,Above Medium
4,Am Gallberg 234,Werne,Nordrhein-Westfalen,Germany,1975-02-05,Partial College,Skilled Manual,1,1,0,0,51804,2017-03-06,1.0,55.41,0.231958,0.533742,-0.682822,1.059589,1,1,low,Above Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18350,3469 Relief Valley Ct.,Royal Oak,British Columbia,Canada,1960-09-06,Bachelors,Clerical,1,1,0,2,87320,2017-03-06,0.0,48.14,0.544379,0.190656,0.474657,-1.055055,0,0,above medium,Low
18351,6126 North Sixth Street,Redmond,Washington,United States,1986-08-15,Graduate Degree,Clerical,0,1,0,0,84374,2017-03-06,1.0,50.23,0.518464,0.289287,0.378646,-0.447131,0,1,Medium,Medium
18352,7182 Olive Hill,Cliffside,British Columbia,Canada,1986-11-14,Graduate Degree,Skilled Manual,0,0,0,0,50961,2017-03-06,1.0,49.56,0.224543,0.257669,-0.710296,-0.642016,0,1,low,Medium
18353,"487, rue Montcalm",Les Ulis,Essonne,France,1983-02-05,Partial High School,Manual,0,1,0,0,33678,2017-03-06,0.0,51.42,0.072511,0.345446,-1.273555,-0.100993,0,1,below low,Medium


In [52]:
# PART-III
# Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively.
# (a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two following objects of your transformed input data.
# (b) Calculate Correlation between two features Commute Distance and Yearly Income
# helpful article
# https://towardsdatascience.com/familiarity-with-coefficients-of-similarity-73697d357acf
# imp diff between 

In [53]:
# Jaccard Similarity = (number of observations in both sets) / (number in either set)
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [54]:

jaccard(df['Male'],df['Single'])

5.448403617740002e-05

In [55]:
# print("Correlation is",np.corrcoef(df['Male'],df['Single']))
# Cosine accepts a 2-D array,so we need to convert the series to numpy array
# 1.Using values method
# 2.Using Pandas.Series.to_numpy()
a=np.array([df['Male'].to_numpy()])
print(a)
b=np.array([df['Single'].values])
print(b)
print('cosine similarity is:\n',cosine_similarity(a,b))


[[1 1 0 ... 0 0 0]]
[[0 0 1 ... 1 1 0]]
cosine similarity is:
 [[0.45939671]]


In [56]:
# b part
# let's calculate correlation between yearlyIncome and avgmonthspend
# np.corrcoef() OR
df['YearlyIncome'].corr(df['AvgMonthSpend'])

0.012202883547392651